In [1]:
import cv2, os, re
import imgproc
import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable
from ResUnet import CRAFT
import torch


net = CRAFT(input_channel = 3, n_classes = 4)

In [2]:
samp = torch.randn((1,3,768,768))

x, output_feature = net(samp)

In [3]:
from config import config
from dataloader import Dataset

cfg = config.Config('./config/config.yml')
dataloader = Dataset(cfg)

/home/jovyan/nas/1_user/eunsung.shin@agilesoda.ai/module/CRAFTS/config/config.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self._dict = yaml.load(self._yaml)


In [ ]:
imgfile = os.path.join(dataloader.img_folder, dataloader.images_path[0])
gtfile = os.path.join(dataloader.gt_folder, dataloader.images_path[0].replace('.jpg', '.txt').replace('.png', '.txt'))
print(imgfile)
print(gtfile)
bboxes, words = dataloader.load_gt(gtfile)


In [ ]:
image = imgproc.loadImage(imgfile)

img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 2380, interpolation=cv2.INTER_LINEAR, mag_ratio= 2.5)
ratio_h = ratio_w = 1 / target_ratio

# preprocessing
x = imgproc.normalizeMeanVariance(img_resized)
x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
x = Variable(x.unsqueeze(0)) 

result, output_feature = net(x)

In [ ]:
result.shape

In [ ]:
STR_inputs = torch.cat([result[:,:2,:,:], output_feature], axis = 1).permute(0,2,3,1).cuda()



In [ ]:
STR_inputs.cpu().detach().numpy()

In [ ]:


def crop_image_by_bbox(image, box, word):

    rot_angle = None
    match = re.findall(r'\[ROT[0-9]+\]', word)
    if len(match) > 0  :
        rot_angle = int(re.findall(r'\d+', match[0])[0])
    word = word.replace('[UNK]', '*')
    word = re.sub(r'[UNK[0-9]+]|[ROT[0-9]+]', '', word)
    if len(box) == 4 :
        w = (int)(np.linalg.norm(box[0] - box[1]))
        h = (int)(np.linalg.norm(box[0] - box[3]))
        width = w
        height = h
        print(width, height)
        if h > w * 1.5 and len(word) != 1 :
            width = h
            height = w
            # [ROT90]일 때
            if rot_angle == 90 or rot_angle == None :
                M = cv2.getPerspectiveTransform(np.float32(box),
                                np.float32(np.array([[0, height], [0,0], [width, 0], [width, height]])))
            if rot_angle == 270 or rot_angle == 0 :
                M = cv2.getPerspectiveTransform(np.float32(box),
                                            np.float32(np.array([[width, 0], [width, height], [0, height], [0, 0]])))
        else:
            M = cv2.getPerspectiveTransform(np.float32(box),
                                            np.float32(np.array([[0, 0], [width, 0], [width, height], [0, height]])))

        warped = cv2.warpPerspective(image, M, (width, height))
        return warped, M
    else : 
        # polygon(>4)
        pts = np.int32(box)
        rect = cv2.boundingRect(pts)
        x,y,w,h = rect
        x,y = max(0, x), max(0, y)
        x,y = min(x, image.shape[1]), min(y, image.shape[0])
        cropped = np.array(image)[y:y+h, x:x+w]
        pts = pts - pts.min(axis = 0)

        mask = np.zeros(cropped.shape[:2], np.uint8)
        cv2.drawContours(mask, [pts], -1, (255,255,255), -1, cv2.LINE_AA)
        cropped_black_bg = cv2.bitwise_and(cropped, cropped, mask=mask)
        bg = np.ones_like(cropped, np.uint8)*255
        cv2.bitwise_not(bg,bg, mask = mask)
        cropped_white_bg = bg + cropped_black_bg

        return cropped_white_bg, (x,y)

In [ ]:
crop_feature, _ = crop_image_by_bbox(np.array(STR_inputs[0].detach()), bboxes[0], words[0])

In [4]:
import torch

data_loader = torch.utils.data.DataLoader(dataloader,
                                              batch_size=cfg.BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=cfg.WORKERS,
                                              drop_last=True,
                                              pin_memory=True)

In [7]:
from tqdm import tqdm
for index, (images, gh_label, gah_label, mask, ori_x, ori_y, word_bboxes_batch, words_batch, words_length_batch) in tqdm(enumerate(data_loader)):
    print(words_batch.shape)
    print(words_length_batch.shape)


1it [00:04,  4.80s/it]

torch.Size([4, 32, 27])
torch.Size([4, 32])


2it [00:12,  6.65s/it]

torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])


6it [00:20,  2.94s/it]

...

torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])


24it [00:28,  1.54it/s]

torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])
torch.Size([4, 32, 27])
torch.Size([4, 32])


25it [00:28,  1.15s/it]


UnboundLocalError: Caught UnboundLocalError in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jovyan/nas/1_user/eunsung.shin@agilesoda.ai/module/CRAFTS/dataloader.py", line 429, in __getitem__
    return self.pull_item(index)
  File "/home/jovyan/nas/1_user/eunsung.shin@agilesoda.ai/module/CRAFTS/dataloader.py", line 653, in pull_item
    og_image, character_bboxes, words, confidence_mask, confidences, word_bboxes, og_shape = self.load_image_gt_and_confidencemask(index)
  File "/

In [ ]:
image, region_scores_torch, affinity_scores_torch, confidence_mask_torch, orientation_x_torch, orientation_y_torch, cropped_word_bboxes, encoded_cropped_words, words_length = dataloader.pull_item( 29)


In [ ]:
from utils import AttnLabelConverter
import data_utils

def load_character_list(path) :
    with open(path, 'r') as f :
        character_list = f.read()
        
    return character_list
character_list = load_character_list('./char_dicts/charset-kor-sp.txt')
converter = AttnLabelConverter(character_list)

pad_batch, text_batch, length_batch = [], [], []
            
idx = 0 
STR_input = STR_inputs[idx]
word_bboxes = word_bboxes_batch[idx]
words = words_batch[idx]
words_length = words_length_batch[idx]
decoded_words = converter.decode(words, words_length)

feature_ls = []
for word_bbox, word, decoded_word, word_length in zip(word_bboxes, words ,decoded_words, words_length) :
    if word_length != 1  :
        cropFeature, _ = data_utils.crop_image_by_bbox(STR_input.detach().numpy(), word_bbox, decoded_word)
        feature_ls.append(cropFeature)

In [ ]:
feature_ls[0]

In [ ]:
from PIL import Image
cv2.resize(STR_input.detach().numpy(), (800, 1200), Image.BICUBIC).shape

In [ ]:
STR_input.shape

In [ ]:
class RegionResizeNormalize(object):

    def __init__(self, size, interpolation=Image.BICUBIC):
        self.size = size
        self.interpolation = interpolation
        self.toTensor = transforms.ToTensor()
        
    def __call__(self, img):
        img = cv2.resize(img, self.size, Image.BICUBIC)
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        return img
    
class RegionNormalizePAD(object):
    def __init__(self, max_size, PAD_type='right'):
        self.toTensor = transforms.ToTensor()
        self.max_size = max_size
        self.max_width_half = math.floor(max_size[2] / 2)
        self.PAD_type = PAD_type

    def __call__(self, img):
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        c, h, w = img.size()
        Pad_img = torch.FloatTensor(*self.max_size).fill_(0) 
        Pad_img[:, :, :w] = img  # right pad
        if self.max_size[2] != w:  # add border Pad
            Pad_img[:, :, w:] = img[ :, :, w - 1].unsqueeze(2).expand(c, h, self.max_size[2] - w)

        return Pad_img
    
    
    
    
class RegionAlignCollate(object) :
    def __init__(self, imgH=32, imgW=100, keep_ratio_with_pad=False):
        self.imgH = imgH
        self.imgW = imgW
        self.keep_ratio_with_pad = keep_ratio_with_pad
    
    def __call__(self, batch):

        batch = filter(lambda x: x is not None, batch)
        
        if self.keep_ratio_with_pad :
            resized_max_w = self.imgW
            input_channel = batch[0].shape[-1]
            transform = RegionNormalizePAD((input_channel, self.imgH, resized_max_w))

            resized_images = []
            for image in batch :
                if math.ceil(self.imgH * ratio) > self.imgW:
                    resized_w = self.imgW
                else:
                    resized_w = math.ceil(self.imgH * ratio)

                resized_image = cv2.resize(image, (resized_w, self.imgH), Image.BICUBIC)
                resized_images.append(transform(resized_image))
                
            image_tensors = torch.cat([t.unsqueeze(0) for t in resized_images], 0)    
            
        else :
            transform = RegionResizeNormalize((self.imgW, self.imgH))
            image_tensors = [transform(image) for image in batch]
            image_tensors = torch.cat([t.unsqueeze(0) for t in image_tensors], 0)
            
        return image_tensors

In [ ]:
padding = RegionAlignCollate(imgH=16, imgW=64, keep_ratio_with_pad = False)

In [ ]:
padding(feature_ls).shape

In [ ]:
cropFeature.shape

In [ ]:
import torchvision.transforms as transforms

tensorfy = transforms.ToTensor()

tensorfy(cropFeature).sub_(0.5).div_(0.5).shape


In [ ]:
pil = Image.open('/home/jovyan/nas/2_public_data/aihub_wildscene_labeled/image/BC255C8EDB6756B8E651FCFF3A43B61C.jpg')

In [ ]:
np.tile(np.array(pil), (3,1,1))

In [ ]:
cv2.resize(np.array(pil), (50,30)).shape

In [ ]:
torch.Tensor([1,2, float('nan')])